In [1]:
import os, re
import time
import random
import requests
from bs4 import BeautifulSoup

In [2]:
def get_soup(url):
    req = requests.get(url, headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
    assert req.status_code == 200, 'request failed'
    html = req.text
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [3]:
def get_urls():
    urls = []
    url_eda = 'https://www.avito.ru/moskva/produkty_pitaniya?p='
    for i in range(101):
        h3s = get_soup(url_eda + str(i)).find_all('h3')
        for i in h3s:
            urls.append(i.find('a')['href'])
        time.sleep(random.randint(1, 4))
    return urls

In [4]:
all_urls = get_urls()
all_urls[:13]

['/moskva/produkty_pitaniya/kartofel_luk_kapusta_s_dostavkoy_opt_i_roznitsa_1642470980',
 '/moskva/produkty_pitaniya/kedrovyy_oreh_dalnevostochnyy_214161856',
 '/moskva/produkty_pitaniya/krasnaya_ikra_1314294440',
 '/moskva/produkty_pitaniya/svechi_s_propolisom_ot_prostatita_i_gemorroya_1685138312',
 '/moskva/produkty_pitaniya/ikra_krasnaya_1583669150',
 '/moskva/produkty_pitaniya/assorti_marinovannoe_433887544',
 '/moskva/produkty_pitaniya/krasnaya_ikra_1063004826',
 '/moskva/produkty_pitaniya/detskoe_pyure_agusha_fruktovoe_1368050928',
 '/moskva/produkty_pitaniya/krasnaya_ikra_1509969940',
 '/moskva/produkty_pitaniya/zheludi_bez_shlyapok_dlya_tvorchestva_1317780568',
 '/moskva/produkty_pitaniya/detskoe_ovoschnoe_pyure_1113340422',
 '/moskva/produkty_pitaniya/chesnok_1596252933',
 '/moskva/produkty_pitaniya/podarochnyy_podarok_1433170629']

У нас есть ссылки на все объявления. Сколько их?

In [6]:
len(all_urls)

5394

In [9]:
def make_corpora(urls):
    if not os.path.exists('./avito_food'):
        os.makedirs('./avito_food')
    for i in urls:
        time.sleep(random.randint(1, 5))
        head_soup = get_soup('http://www.avito.ru' + i).find('div', attrs={'class':'sticky-header-prop sticky-header-title'})
        if head_soup == None:
            head = 'нет заголовка'
        else:
            head = head_soup.text
        date_soup = get_soup('http://www.avito.ru' + i).find('div', attrs={'class':'title-info-metadata-item'})
        if date_soup == None:
            date =  'нет информации о дате'
        else:
            date = date_soup.text
        text_soup = get_soup('http://www.avito.ru' + i).find('div', attrs={'class':'item-description-text'})
        if text_soup == None:
            text = 'нет текста объявления'
        else:
            text = text_soup.text
        price_soup = get_soup('http://www.avito.ru' + i).find('span', attrs={'itemprop':'price'})
        if price_soup == None:
            price = 'нет информации о цене'
        else:
            price = price_soup.text
        full_info = head + '\n' + date + '\n' + text + '\n' + price
        with open ('./avito_food/' + i.replace(r'/', '=') + '.txt', 'w', encoding = 'utf-8') as f:
            f.write(full_info)
    return

In [10]:
make_corpora(all_urls)